In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\agost\\Data_Career\\End_to_end_projects\\Abalone_Age_Prediction'

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    local_data_file: Path
    trained_model_path: Path
    param_target_col: str
    elastic_pickle: Path
    lasso_pickle: Path
    lr_pickle: Path
    rfr_pickle: Path
    ridge_pickle: Path
    svr_pickle: Path

In [7]:
from Abalone_Age_Prediction.utils.common import create_directories, read_yaml, load_object
from Abalone_Age_Prediction.constants import *


In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #the artifacts_root is the key of the dictionary created
                                                # in the yaml file and we can read this key like that instead of
                                                # ["artifacts_root"] because we used the ConfigBox in the common.py file


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation #model evaluation is the other key value of the dictionary in the config.yaml file

        create_directories([config.root_dir,config.save_training_file])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            local_data_file = config.local_data_file,
            trained_model_path = config.trained_model_path,
            param_target_col=self.params.TARGET,
            elastic_pickle= config.elastic_pickle,
            lasso_pickle= config.lasso_pickle,
            lr_pickle= config.lr_pickle,
            rfr_pickle= config.rfr_pickle,
            ridge_pickle= config.ridge_pickle,
            svr_pickle= config.svr_pickle
        )                                     

        return model_evaluation_config

In [9]:
import pandas as pd
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [12]:
class ModelPreparationTraining:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    
    def read_file(self):
        """
        Read the csv file
        """
        try:
            
            testing_data = pd.read_csv(self.config.local_data_file)

            return testing_data

        except Exception as e:
            e

    def evaluation_metrics(self,X_test,y_test, models):
        """
        Evaluate all the models and returns 3 dictionaries with the different evaluation scores
        """
        


    def model_evaluation(self,testing_data):
        """
        Loading the data and the models and evaluate them
        """
        X_test = testing_data.drop(labels = self.config.param_target_col, axis = 1)
        y_test = testing_data[self.config.param_target_col]

        models = {
            "elastic" : load_object(Path(self.config.elastic_pickle)),
            "lasso" : load_object(Path(self.config.lasso_pickle)),
            "lr" : load_object(Path(self.config.lr_pickle)),
            "rfr" : load_object(Path(self.config.rfr_pickle)),
            "ridge" : load_object(Path(self.config.ridge_pickle)),
            "svr" : load_object(Path(self.config.svr_pickle))
        }

        



In [15]:
dic = {"key1" : 1,
       "key2" : 2}

for key in dic.keys():
    print(dic[key])

1
2
